# Web Scraping

### Import Dependencies 

In [1]:
!apt update
!pip install selenium
!pip install webdriver-manager

'apt' is not recognized as an internal or external command,
operable program or batch file.


In [1]:
import random
import time
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium import webdriver

#因为我用的edge浏览器，所以这里用的是edge driver，如果你电脑里装的是chrome，要换成chrome driver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
info_df = pd.read_csv("../tmdb_5000_data/tmdb_5000_movies.csv")
info_df = info_df.drop_duplicates()
info_df = info_df[['id', 'title','release_date']]
info_df['release_date'] = pd.to_datetime(info_df['release_date'], format = "%Y-%m-%d").dt.strftime('%Y')
info_df.rename(columns= {'id':'movie_id', 'release_date':'startYear'}, inplace = True)
info_df["storyline"] = np.nan
info_df['storyline'] = info_df['storyline'].astype('object')
info_df.head(10)

,movie_id,title,startYear,storyline
0,19995,Avatar,2009,NaN
1,285,Pirates of the Caribbean: At World's End,2007,NaN
2,206647,Spectre,2015,NaN
3,49026,The Dark Knight Rises,2012,NaN
4,49529,John Carter,2012,NaN
5,559,Spider-Man 3,2007,NaN
6,38757,Tangled,2010,NaN
7,99861,Avengers: Age of Ultron,2015,NaN
8,767,Harry Potter and the Half-Blood Prince,2009,NaN
9,209112,Batman v Superman: Dawn of Justice,2016,NaN


In [3]:
imdb_df = pd.read_csv('../tmdb_5000_data/title.basics.tsv', sep = '\t')
imdb_df = imdb_df[imdb_df['titleType'] == 'movie']
imdb_df = imdb_df[['tconst', 'primaryTitle','startYear']]
imdb_df['startYear'] = imdb_df['startYear'].replace('\n','').astype(str)
imdb_df.rename(columns= {'tconst':'imdb_id', 'primaryTitle':'title'}, inplace = True)
imdb_df.head(10)

C:\Users\czhao\AppData\Local\Temp\ipykernel_25508\1768068457.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  imdb_df = pd.read_csv('../tmdb_5000_data/title.basics.tsv', sep = '\t')


,imdb_id,title,startYear
8,tt0000009,Miss Jerry,1894
144,tt0000147,The Corbett-Fitzsimmons Fight,1897
498,tt0000502,Bohemios,1905
570,tt0000574,The Story of the Kelly Gang,1906
587,tt0000591,The Prodigal Son,1907
610,tt0000615,Robbery Under Arms,1907
625,tt0000630,Hamlet,1908
668,tt0000675,Don Quijote,1908
672,tt0000679,The Fairylogue and Radio-Plays,1908
828,tt0000838,A Cultura do Cacau,1909


In [4]:
full_df = info_df.merge(imdb_df, how = 'inner',on = ['title', 'startYear'])
full_df = full_df.reset_index()
full_df.head()

,index,movie_id,title,startYear,storyline,imdb_id
0,0,19995,Avatar,2009,NaN,tt0499549
1,1,285,Pirates of the Caribbean: At World's End,2007,NaN,tt0449088
2,2,206647,Spectre,2015,NaN,tt2379713
3,3,49026,The Dark Knight Rises,2012,NaN,tt1345836
4,4,49529,John Carter,2012,NaN,tt0401729


### Define url function

In [5]:
def get_url(movie_name):
    url_template = "https://www.imdb.com/title/{}/"
    url = url_template.format(movie_name)
    return url

### Scrape movie reviews from critics

In [6]:
import re
from selenium.common.exceptions import NoSuchElementException

#create driver instance.
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

#iterate through index, reverse() is called so it is easier to see the result and debug. 
indexes = full_df.index.tolist()
movie_num=0
for i in indexes:
    movie_num += 1
    name_og = full_df['title'].iloc[i]
    imdb_id = full_df['imdb_id'].iloc[i]

    driver.get(get_url(imdb_id)) #The entire website's html file
    driver.implicitly_wait(5)     #Wait time to avoid human verification, 
    #storylines = driver.find_elements(By.CLASS_NAME, 'ipc-html-content-inner-div') #Find specific element/elements by class name
    #story_collections = []
    #for storyline in storylines:
    #    result_html = storyline.get_attribute('innerHTML')
    #    soup = BeautifulSoup(result_html, 'html.parser') #Process the driver-returned elements.

    #    story_collections.append(str(soup).strip())

    #full_df.at[i, 'storyline'] = story_collections[0]
    xpath = '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[@data-testid="Storyline"]/div[2]/div[1]/div/div'
    storyline = ''
    try:
        storyline = driver.find_element(By.XPATH, xpath)
        result_html = storyline.get_attribute('innerHTML')
        soup = BeautifulSoup(result_html, 'html.parser')
        storyline = str(soup).strip().split("<span")[0]
    except NoSuchElementException:
        pass
    full_df.at[i, 'storyline'] = storyline
    print("{}: story added for movie: {}".format(movie_num, name_og))
driver.quit()

[WDM] - Downloading: 100%|████████████████████████████████████████████████████████| 6.58M/6.58M [00:00<00:00, 11.2MB/s]


1: story added for movie: Avatar
2: story added for movie: Pirates of the Caribbean: At World's End
3: story added for movie: Spectre
4: story added for movie: The Dark Knight Rises
5: story added for movie: John Carter
6: story added for movie: Spider-Man 3
7: story added for movie: Tangled
8: story added for movie: Avengers: Age of Ultron
9: story added for movie: Harry Potter and the Half-Blood Prince
10: story added for movie: Batman v Superman: Dawn of Justice
11: story added for movie: Superman Returns
12: story added for movie: Quantum of Solace
13: story added for movie: Pirates of the Caribbean: Dead Man's Chest
14: story added for movie: The Lone Ranger
15: story added for movie: Man of Steel
16: story added for movie: The Chronicles of Narnia: Prince Caspian
17: story added for movie: The Avengers
18: story added for movie: Pirates of the Caribbean: On Stranger Tides
19: story added for movie: Men in Black 3
20: story added for movie: The Hobbit: The Battle of the Five Armie

175: story added for movie: The Revenant
176: story added for movie: Turbo
177: story added for movie: Rango
178: story added for movie: Penguins of Madagascar
179: story added for movie: The Bourne Ultimatum
180: story added for movie: Kung Fu Panda
181: story added for movie: Ant-Man
182: story added for movie: The Hunger Games: Catching Fire
183: story added for movie: Home
184: story added for movie: Home
185: story added for movie: War of the Worlds
186: story added for movie: Bad Boys II
187: story added for movie: Puss in Boots
188: story added for movie: Salt
189: story added for movie: Noah
190: story added for movie: The Adventures of Tintin
191: story added for movie: Harry Potter and the Prisoner of Azkaban
192: story added for movie: Australia
193: story added for movie: After Earth
194: story added for movie: Dinosaur
195: story added for movie: Night at the Museum: Secret of the Tomb
196: story added for movie: Megamind
197: story added for movie: R.I.P.D.
198: story add

352: story added for movie: Sherlock Holmes
353: story added for movie: Ben-Hur
354: story added for movie: Atlantis: The Lost Empire
355: story added for movie: Alvin and the Chipmunks: The Road Chip
356: story added for movie: Valkyrie
357: story added for movie: You Don't Mess with the Zohan
358: story added for movie: Pixels
359: story added for movie: A.I. Artificial Intelligence
360: story added for movie: The Haunted Mansion
361: story added for movie: Contact
362: story added for movie: Hollow Man
363: story added for movie: The Interpreter
364: story added for movie: Percy Jackson: Sea of Monsters
365: story added for movie: Lara Croft Tomb Raider: The Cradle of Life
366: story added for movie: Now You See Me 2
367: story added for movie: The Saint
368: story added for movie: Spy Game
369: story added for movie: Mission to Mars
370: story added for movie: Rio
371: story added for movie: Bicentennial Man
372: story added for movie: Volcano
373: story added for movie: The Devil'

C:\Users\czhao\anaconda3\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


...

3638: story added for movie: Stake Land
3639: story added for movie: The Last Time I Committed Suicide
3640: story added for movie: Futuro Beach
3641: story added for movie: Another Happy Day
3642: story added for movie: A Lonely Place to Die
3643: story added for movie: Nothing
3644: story added for movie: The Geographer Drank His Globe Away
3645: story added for movie: 1776
3646: story added for movie: Inescapable
3647: story added for movie: Hell's Angels
3648: story added for movie: Purple Violets
3649: story added for movie: The Veil
3650: story added for movie: The Loved Ones
3651: story added for movie: The Perfect Wave
3652: story added for movie: A Man for All Seasons
3653: story added for movie: Network
3654: story added for movie: Gone with the Wind
3655: story added for movie: Desert Dancer
3656: story added for movie: Major Dundee
3657: story added for movie: Annie Get Your Gun
3658: story added for movie: Four Lions
3659: story added for movie: Defendor
3660: story added 

3824: story added for movie: Fat, Sick & Nearly Dead
3825: story added for movie: A Haunted House
3826: story added for movie: 2016: Obama's America
3827: story added for movie: That Thing You Do!
3828: story added for movie: Halloween III: Season of the Witch
3829: story added for movie: Escape from the Planet of the Apes
3830: story added for movie: Hud
3831: story added for movie: Kevin Hart: Let Me Explain
3832: story added for movie: My Own Private Idaho
3833: story added for movie: Garden State
3834: story added for movie: Before Sunrise
3835: story added for movie: Jesus' Son
3836: story added for movie: Saving Face
3837: story added for movie: Brick Lane
3838: story added for movie: Robot & Frank
3839: story added for movie: My Life Without Me
3840: story added for movie: The Spectacular Now
3841: story added for movie: Religulous
3842: story added for movie: Fuel
3843: story added for movie: Fuel
3844: story added for movie: Valley of the Heart's Delight
3845: story added for 

4001: story added for movie: Out of the Dark
4002: story added for movie: The Bubble
4003: story added for movie: The Conversation
4004: story added for movie: Mississippi Mermaid
4005: story added for movie: I Love Your Work
4006: story added for movie: Cabin Fever
4007: story added for movie: Waitress
4008: story added for movie: Bloodsport
4009: story added for movie: Mr. Smith Goes to Washington
4010: story added for movie: Kids
4011: story added for movie: The Squid and the Whale
4012: story added for movie: Kissing Jessica Stein
4013: story added for movie: Kickboxer: Vengeance
4014: story added for movie: Spellbound
4015: story added for movie: Exotica
4016: story added for movie: Buffalo '66
4017: story added for movie: Insidious
4018: story added for movie: Repo Man
4019: story added for movie: Nine Queens
4020: story added for movie: The Gatekeepers
4021: story added for movie: The Ballad of Jack and Rose
4022: story added for movie: The To Do List
4023: story added for movie

4180: story added for movie: Bang Bang Baby
4181: story added for movie: Fear Clinic
4182: story added for movie: Zombie Hunter
4183: story added for movie: A Fine Step
4184: story added for movie: Charly
4185: story added for movie: Banshee Chapter
4186: story added for movie: Ask Me Anything
4187: story added for movie: And Then Came Love
4188: story added for movie: Food Chains
4189: story added for movie: On the Waterfront
4190: story added for movie: 4 Months, 3 Weeks and 2 Days
4191: story added for movie: The Horror Network Vol. 1
4192: story added for movie: Hard Candy
4193: story added for movie: The Quiet
4194: story added for movie: Circumstance
4195: story added for movie: Fruitvale Station
4196: story added for movie: The Brass Teapot
4197: story added for movie: Bambi
4198: story added for movie: The Hammer
4199: story added for movie: Latter Days
4200: story added for movie: Elza
4201: story added for movie: 1982
4202: story added for movie: For a Good Time, Call...
4203

4361: story added for movie: American Desi
4362: story added for movie: Amidst the Devil's Wings
4363: story added for movie: Cube
4364: story added for movie: Love and Other Catastrophes
4365: story added for movie: November
4366: story added for movie: Like Crazy
4367: story added for movie: Teeth and Blood
4368: story added for movie: Sugar Town
4369: story added for movie: The Motel
4370: story added for movie: The Canyons
4371: story added for movie: On the Outs
4372: story added for movie: Shotgun Stories
4373: story added for movie: Exam
4374: story added for movie: The Sticky Fingers of Time
4375: story added for movie: Rust
4376: story added for movie: Ink
4377: story added for movie: The Christmas Bunny
4378: story added for movie: UnDivided
4379: story added for movie: The Frozen
4380: story added for movie: Give Me Shelter
4381: story added for movie: The Big Parade
4382: story added for movie: Little Big Top
4383: story added for movie: Along the Roadside
4384: story added

In [7]:
full_df

,index,movie_id,title,startYear,storyline,imdb_id
0,0,19995,Avatar,2009,"When his brother is killed in a robbery, parap...",tt0499549
1,1,285,Pirates of the Caribbean: At World's End,2007,"After Elizabeth, Will, and Captain Barbossa re...",tt0449088
2,2,206647,Spectre,2015,A cryptic message from the past sends James Bo...,tt2379713
3,3,49026,The Dark Knight Rises,2012,Despite his tarnished reputation after the eve...,tt1345836
4,4,49529,John Carter,2012,"John Carter, a Civil War veteran, who in 1868 ...",tt0401729
...,...,...,...,...,...,...
4499,4499,14337,Primer,2004,"Engineers Aaron, Abe, Robert and Phillip are w...",tt0390384
4500,4500,67238,Cavite,2005,"Adam, a security guard, travels from Californi...",tt0428303
4501,4501,9367,El Mariachi,1992,El Mariachi just wants to play his guitar and ...,tt0104815
4502,4502,72766,Newlyweds,2011,Newlyweds Buzzy and Katie find their blissful ...,tt1880418


### Save results

In [8]:
# Convert the dataframe to a csv file
date = datetime.today().strftime('%Y-%m-%d')
full_df.to_csv(date + "movie_with_imdb_storyline.csv", index=False)

In [9]:
# read csv 
full_df = pd.read_csv(date +"movie_with_imdb_storyline.csv")
full_df

,index,movie_id,title,startYear,storyline,imdb_id
0,0,19995,Avatar,2009,"When his brother is killed in a robbery, parap...",tt0499549
1,1,285,Pirates of the Caribbean: At World's End,2007,"After Elizabeth, Will, and Captain Barbossa re...",tt0449088
2,2,206647,Spectre,2015,A cryptic message from the past sends James Bo...,tt2379713
3,3,49026,The Dark Knight Rises,2012,Despite his tarnished reputation after the eve...,tt1345836
4,4,49529,John Carter,2012,"John Carter, a Civil War veteran, who in 1868 ...",tt0401729
...,...,...,...,...,...,...
4499,4499,14337,Primer,2004,"Engineers Aaron, Abe, Robert and Phillip are w...",tt0390384
4500,4500,67238,Cavite,2005,"Adam, a security guard, travels from Californi...",tt0428303
4501,4501,9367,El Mariachi,1992,El Mariachi just wants to play his guitar and ...,tt0104815
4502,4502,72766,Newlyweds,2011,Newlyweds Buzzy and Katie find their blissful ...,tt1880418
